In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.onnx
import torch
import onnx
import numpy as np
import onnxruntime as ort
from workload_generator import parse_onnx, write_csv



/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load the pretrained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Example usage
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


# Set the path for saving the ONNX file
onnx_file_path = "model.onnx"

# Set the input tensor
input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30]])

# Export the model to ONNX
torch.onnx.export(model, input_ids, onnx_file_path)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, how are you?

I'm a little bit of a nerd. I'm a


In [3]:
# Load the ONNX model
onnx_model = onnx.load("model.onnx")


In [4]:
# Parse the ONNX model
parsed_model, other_ops = parse_onnx(onnx_model)

In [8]:
pdict = {k: v for k,v in parsed_model.items() if v.op_type == 'Gemm' or v.op_type == 'MatMul'}
pdict['transformer.h.1.attn.MatMul']

ParsedModel(op_type='MatMul', input_shapes=[(1, 12), (12, 12)], output_shapes=[(1, 12)], macs=144, attribute=[], param_shapes=[])

In [6]:
# Write the parsed model to a CSV file
write_csv(parsed_model, "gpt2_gen.csv")

Duplicate found: transformer.h.0.attn.MatMul_1
Duplicate found: transformer.h.1.attn.c_attn.Gemm
Duplicate found: transformer.h.1.attn.MatMul
Duplicate found: transformer.h.1.attn.MatMul_1
Duplicate found: transformer.h.1.attn.c_proj.Gemm
Duplicate found: transformer.h.1.mlp.c_fc.Gemm
Duplicate found: transformer.h.1.mlp.c_proj.Gemm
Duplicate found: transformer.h.2.attn.c_attn.Gemm
Duplicate found: transformer.h.2.attn.MatMul
Duplicate found: transformer.h.2.attn.MatMul_1
Duplicate found: transformer.h.2.attn.c_proj.Gemm
Duplicate found: transformer.h.2.mlp.c_fc.Gemm
Duplicate found: transformer.h.2.mlp.c_proj.Gemm
Duplicate found: transformer.h.3.attn.c_attn.Gemm
Duplicate found: transformer.h.3.attn.MatMul
Duplicate found: transformer.h.3.attn.MatMul_1
Duplicate found: transformer.h.3.attn.c_proj.Gemm
Duplicate found: transformer.h.3.mlp.c_fc.Gemm
Duplicate found: transformer.h.3.mlp.c_proj.Gemm
Duplicate found: transformer.h.4.attn.c_attn.Gemm
Duplicate found: transformer.h.4.attn.M